In [70]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np

from sklearn import decomposition  
from sklearn.preprocessing import scale  
from sklearn import preprocessing 
from sklearn import linear_model
from sklearn import model_selection
#from sklearn import cross_validation

from scipy.stats import boxcox
from scipy.stats import spearmanr
from scipy.stats import pearsonr

### Importar Datos

## 2011

df_vivienda_2011 = pd.read_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\01  - Precio Vivienda\Distritos\2011_Distritos_PrecioVivienda_V2.csv", sep=";")
df_ocupacion_2011 = pd.read_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Secciones\2011_Secciones_Ocupacion_V2.csv", sep=";")
df_estudios_2011 = pd.read_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\04  - Estudios\Secciones\2011_Secciones_Estudios_Detalle_V2.csv", sep=";")

## 2016

df_vivienda_2021 = pd.read_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\01  - Precio Vivienda\Distritos\2021_Distritos_PrecioVivienda_V2.csv", sep=";")
df_ocupacion_2021 = pd.read_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Secciones\2021_Secciones_Ocupacion_V2.csv", sep=";")
df_estudios_2021 = pd.read_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\04  - Estudios\Secciones\2021_Secciones_Estudios_Detalle_V2.csv", sep=";")

path_scores = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Scores"
path_analytical = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Analytical"

#Para poder unir los valores de vivienda que solo estan por distrito con las secciones que son los datos de ocupacion y estudios se crea la columna NUMERO en estos dos
df_ocupacion_2011['NUMERO'] = df_ocupacion_2011['NUMSECCENS'].astype(str).str[:-5].astype(float)
df_ocupacion_2021['NUMERO'] = df_ocupacion_2021['NUMSECCENS'].astype(str).str[:-5].astype(float)
df_estudios_2011['NUMERO'] = df_estudios_2011['NUMSECCENS'].astype(str).str[:-5].astype(float)
df_estudios_2021['NUMERO'] = df_estudios_2021['NUMSECCENS'].astype(str).str[:-5].astype(float)

#Eliminar Martiricos-La Roca porque no hay datos de 2011 en vivienda
df_estudios_2011 = df_estudios_2011[df_estudios_2011['NUMERO'] != 5]
df_estudios_2021 = df_estudios_2021[df_estudios_2021['NUMERO'] != 5]
df_ocupacion_2011 = df_ocupacion_2011[df_ocupacion_2011['NUMERO'] != 5]
df_ocupacion_2021 = df_ocupacion_2021[df_ocupacion_2021['NUMERO'] != 5]
df_vivienda_2011 = df_vivienda_2011[df_vivienda_2011['NUMERO'] != 5]
df_vivienda_2021 = df_vivienda_2021[df_vivienda_2021['NUMERO'] != 5]

#Unir estudios y ocupacion por la seccion
df_estudios_ocupacion_2011 = pd.merge(df_estudios_2011, df_ocupacion_2011, on='NUMSECCENS', how='inner')[["NUMSECCENS","NUMERO_x","kw_pct","he_pct"]]
df_estudios_ocupacion_2021 = pd.merge(df_estudios_2021, df_ocupacion_2021, on='NUMSECCENS', how='inner')[["NUMSECCENS","NUMERO_x","kw_pct","he_pct"]]
df_estudios_ocupacion_2011.rename(columns={'NUMERO_x': 'NUMERO'}, inplace=True)
df_estudios_ocupacion_2021.rename(columns={'NUMERO_x': 'NUMERO'}, inplace=True)
print(df_estudios_ocupacion_2011.count())
print(df_estudios_ocupacion_2021.count())

#Unir vivienda y (estudios-ocupacion) por distrito
df_vivienda_estudios_ocupacion_2011 = pd.merge(df_estudios_ocupacion_2011, df_vivienda_2011, on='NUMERO', how='left')
df_vivienda_estudios_ocupacion_2021 = pd.merge(df_estudios_ocupacion_2021, df_vivienda_2021, on='NUMERO', how='left')
print(df_vivienda_estudios_ocupacion_2011.count())
print(df_vivienda_estudios_ocupacion_2021.count())
print(df_vivienda_estudios_ocupacion_2011.head())
print(df_vivienda_estudios_ocupacion_2021.head())

df11 = df_vivienda_estudios_ocupacion_2011.set_index('NUMSECCENS')[["he_pct","kw_pct","median_price_inf"]]
df21 = df_vivienda_estudios_ocupacion_2011.set_index('NUMSECCENS')[["he_pct","kw_pct","median_price_inf"]]

checks = {
    "Qualifications 2011":df11["he_pct"],
    "Qualifications 2021":df21["he_pct"],
    "Occupations 2011":df11["kw_pct"],
    "Occupations 2021":df21["kw_pct"],
    "House Prices 2011":df11["median_price_inf"],
    "House Prices 2021":df21["median_price_inf"],
}

for k, v in checks.items():
    if (np.isnan(v.values).any()):
        print("Have null values in data set: " + k)

#  Create dataset of indicator data - 2001
res_11 = pd.concat([df11["he_pct"],df11["kw_pct"],df11["median_price_inf"]], axis=1)
res_21 = pd.concat([df21["he_pct"],df21["kw_pct"],df21["median_price_inf"]], axis=1)

# Create dataset of indicator data
X_11 = res_11.values
X_21 = res_21.values

#  Join 2001 and 2011 datasets and sanity-check
SES_inds = np.concatenate((X_11, X_21), axis=0)

#print(SES_inds)

print("Any infinite values? " + str(~np.isfinite(SES_inds).any()))
print("Any NaN values? " + str(np.isnan(SES_inds).any()))

#  Median removal and Unit scaling
scaler = preprocessing.RobustScaler()
scaler.fit(SES_inds)
SES_inds = scaler.transform(SES_inds)

print("Data scaled and transformed.")

pca_full = decomposition.PCA()                           # Use all Principal Components
pca_full.fit(SES_inds)                                   # Train model on data
SES_full_T = pd.DataFrame(pca_full.transform(SES_inds))  # Transform data using model

print("The amount of explained variance of the SES score using each component is...")
print(pca_full.explained_variance_ratio_)

# Adapted from https://stackoverflow.com/questions/22984335/recovering-features-names-of-explained-variance-ratio-in-pca-with-sklearn
i = np.identity(SES_inds.shape[1])  # identity matrix

coef = pca_full.transform(i)

loadings = pd.DataFrame(coef, index=res_11.columns)
loadings.to_csv(os.path.join(path_scores,"NoTransform" + '-Loadings-Secciones-2021.csv.gz'), compression='gzip', index=True, sep=";")

#  Fitting PCA Model to derive SES score
pca = decomposition.PCA(n_components=1)             # Only need 1st Principal Component
pca.fit(SES_inds)                                   #  Train model on data
SES_inds_T = pd.DataFrame(pca.transform(SES_inds))  #  Transform data using model

print("The amount of explained variance of the SES score is: {0:6.5f}".format(pca.explained_variance_ratio_[0]))

#  Split transformed data into 2011 and 2021 datasets
#  Note the way we do this to deal with missing data (if any)
scores_11 = SES_inds_T.loc[0:len(X_11)-1,0]
scores_21 = SES_inds_T.loc[len(X_21):,0]

# Create dfs from the two sets of scores
res_11 = res_11.assign(scores=pd.Series(scores_11).values)
res_21 = res_21.assign(scores=pd.Series(scores_21).values)

#res.columns = ['LSOANM','PRICE-01','QUALS-01','OCC-01','INCOME-01','PRICE-11',
#               'QUALS-11','OCC-11','INCOME-11','SES_01','SES_11']

# Join them together so we've got a single df for 2011 and 2021
res = res_11.merge(res_21, how='outer', suffixes=('_11','_21'), left_index=True, right_index=True)

# Rename columns for consistency with Jordan's code
res.rename(columns={'scores_11':'SES_11', 'scores_21':'SES_21'}, inplace=True)

# Sanity check
print(res.head(9))

#  Compute rank of LSOA in 2011 (so low rank = 'low status')
res['RANK_11'] = res.SES_11.rank(ascending=False)

#  Compute rank of LSOA in 2021 (so low rank = 'low status')
res['RANK_21'] = res.SES_21.rank(ascending=False)

#  Compute amount by which LSOA has ascended (so +ve = status improvement; -ve = status decline)
res.loc[:,'SES_ASC'] = res.loc[:,'SES_21'] - res.loc[:,'SES_11']

import re 
#  Calculate LSOA percentile score in 01
res.loc[:,'SES_PR_11'] = res.RANK_11.rank(ascending=False, pct=True) * 100

#  Calculate LSOA percentile score in 11
res.loc[:,'SES_PR_21'] = res.RANK_21.rank(ascending=False, pct=True) * 100

#  Calculate percentile change (so +ve = 'moved up' in the world; -ve = 'moved down')
res.loc[:,'SES_PR_ASC'] = res.loc[:,'SES_PR_21'] - res.loc[:,'SES_PR_11']

inp = res.loc[:,[x for x in res.columns if 'SES' not in x and 'RANK' not in x]]

# Tidy up the naming
inp.rename(columns=lambda x: re.sub('_21',' 2021',re.sub('_11',' 2011',x)), inplace=True)
inp.rename(columns=lambda x: re.sub('kw_pct','Knowledge Worker Percentage',x), inplace=True)
inp.rename(columns=lambda x: re.sub('he_pct','Highly-Educated Percentage',x), inplace=True)
inp.rename(columns=lambda x: re.sub('median_price_inf','Property Prices (Transformed)',x), inplace=True)

# Save to file (note that we are also saving some info about the input variables as we use these as well)
res[
    ['RANK_11','RANK_21','SES_11','SES_21','SES_ASC','SES_PR_11','SES_PR_21','SES_PR_ASC']
].to_csv(os.path.join(path_analytical,"No transform" + '-Scores-Secciones.csv.gz'), compression='gzip', index=True, sep=";") 
inp[
    [x for x in inp.columns if '2011' in x]
].to_csv(os.path.join(path_scores,"No transform" + '-Inputs-Secciones-2011.csv.gz'), compression='gzip', index=True, sep=";")
inp[
    [x for x in inp.columns if '2021' in x]
].to_csv(os.path.join(path_scores,"No transform" + '-Inputs-Secciones-2021.csv.gz'), compression='gzip', index=True, sep=";")

res.to_csv(os.path.join(path_scores,"Scores-Secciones.csv"),index=True, sep=";")

NUMSECCENS    425
NUMERO        425
kw_pct        425
he_pct        425
dtype: int64
NUMSECCENS    135
NUMERO        135
kw_pct        135
he_pct        135
dtype: int64
NUMSECCENS          425
NUMERO              425
kw_pct              425
he_pct              425
NOMBRE              425
median_price_inf    425
dtype: int64
NUMSECCENS          135
NUMERO              135
kw_pct              135
he_pct              135
NOMBRE              135
median_price_inf    135
dtype: int64
   NUMSECCENS  NUMERO    kw_pct    he_pct  NOMBRE  median_price_inf
0      1001.0     1.0  0.410302  0.374126  CENTRO       2361.290759
1      1002.0     1.0  0.899441  0.832168  CENTRO       2361.290759
2      1003.0     1.0  0.924686  0.903226  CENTRO       2361.290759
3      1004.0     1.0  0.305556  0.357447  CENTRO       2361.290759
4      2001.0     2.0  0.333333  0.812500    ESTE       2846.996138
   NUMSECCENS  NUMERO    kw_pct    he_pct  NOMBRE  median_price_inf
0      1001.0     1.0  0.576744  0.41304